In [1]:
# Install required packages
!pip install -q \
    "pyarrow<15.0.0" \
    transformers \
    datasets \
    tokenizers \
    seqeval \
    tensorflow_probability --upgrade \
    evaluate \
    hyperopt \
    "ray[tune]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 

In [2]:
# Load libraries
import torch
from datasets import load_from_disk, DatasetDict

from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer
)

import numpy as np
import evaluate
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer

In [4]:
music = load_from_disk("/content/drive/MyDrive/Code/hfdata_music.json")

In [5]:
# Set model
model_checkpoint = "bert-base-cased"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Create tokenized dataset
tokenized_music = music.map(tokenize_and_align_labels, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
# Show example of tokenized input sequence
example = music["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'suggestions', 'for', 'metal', 'mixed', 'with', 'classical', 'o', '##ches', '##tra', '[SEP]']


In [7]:
# Initialise data_collator for token classification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Load seqeval
seqeval = evaluate.load("seqeval")

# Map and define text and numeral labels
id2label = {
    0: "O",
    1: "B-Artist",
    2: "I-Artist",
    3: "B-WoA",
    4: "I-WoA",
}

label2id = {
    "O": 0,
    "B-Artist": 1,
    "I-Artist": 2,
    "B-WoA": 3,
    "I-WoA": 4
}

label_list = ["O","B-Artist","I-Artist","B-WoA","I-WoA"]
labels = [label_list[i] for i in example[f"ner_tags"]]

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(p.predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    mlb = MultiLabelBinarizer()
    true_labels = mlb.fit_transform(true_labels)
    true_predictions = mlb.transform(true_predictions)

    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(true_labels, true_predictions, average='macro')
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(true_labels, true_predictions, average='micro')
    accuracy = accuracy_score(true_labels, true_predictions)

    return {
        'macro_precision': precision_macro,
        'macro_recall': recall_macro,
        'macro_f1': f1_macro,
        'micro_precision': precision_micro,
        'micro_recall': recall_micro,
        'micro_f1': f1_micro,
        'accuracy': accuracy
    }

In [8]:
# Initialise model
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
)

# Train model with standard parameters
training_args = TrainingArguments(
    "bert-finetuned-ner",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_music["train"],
    eval_dataset=tokenized_music["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Macro Precision,Macro Recall,Macro F1,Micro Precision,Micro Recall,Micro F1,Accuracy
1,No log,0.287780,0.872215,0.757181,0.799525,0.911877,0.830716,0.869406,0.661538
2,No log,0.249549,0.887798,0.776474,0.820937,0.922053,0.846422,0.882621,0.734615


TrainOutput(global_step=224, training_loss=0.3259296417236328, metrics={'train_runtime': 26.9945, 'train_samples_per_second': 132.249, 'train_steps_per_second': 8.298, 'total_flos': 53260051827060.0, 'train_loss': 0.3259296417236328, 'epoch': 2.0})

In [10]:
# Evaluate test dataset and print results.
res = trainer.evaluate(tokenized_music["test"])

for metric_name, metric_value in res.items():
    print(f"{metric_name}: {metric_value:.4f}")

eval_loss: 0.3097
eval_macro_precision: 0.8622
eval_macro_recall: 0.7504
eval_macro_f1: 0.7923
eval_micro_precision: 0.9086
eval_micro_recall: 0.8281
eval_micro_f1: 0.8665
eval_accuracy: 0.6937
eval_runtime: 0.4364
eval_samples_per_second: 1159.4830
eval_steps_per_second: 73.3270
epoch: 2.0000
